# Load modules

In [1]:
import sys
sys.path.append("./modules")
from datetime import datetime
from IPython.display import HTML
# Loading cleaning-merging and aggregating-transforming classes
from clean_merge import *
from aggregate_transform import *

# Set date to create dataset for

In [2]:
#dates = ["2018-01-01", "2017-01-01", "2016-01-01", "2015-01-01", "2014-01-01", "2013-01-01"]
#list_date_dataset = [datetime.strptime(date, '%Y-%m-%d') for date in dates]
date_dataset = datetime.strptime("2018-01-01", '%Y-%m-%d')

# Setup cleaner-merger and aggregator-transformer

In [3]:
project = 'graydon-moving-indicator'
bucket = 'graydon-data'
# Create object for cleaning and merging
clean_merge = Clean_Merge(name_project = project, name_bucket = bucket, 
                          dir_input_data = '01_input', dir_output_data = '02_cleaned')
# Create object for aggregating and transforming
aggregate_transform = Aggregate_Transform(name_project = project, name_bucket = bucket,
                                          dir_input_data = '02_cleaned', dir_output_data = '03_aggregated')

# Executing cleaning-merging, aggregating-transforming

In [ ]:
df_month_year = clean_merge.clean_merge_data(date_dataset)

In [ ]:
import feather
dir_output = clean_merge.dir_output_data
file_output = "cleaned_merged_" + date_dataset.strftime('%Y-%m-%d') + ".feather"

file_path = dir_output + '/' + file_output
df_month_year.to_feather(file_path)

In [4]:
df = aggregate_transform.get_merged_data(date_dataset)

cleaned_merged_2018-01-01.csv
02_cleaned/cleaned_merged_2018-01-01.csv


/home/mrodriguezlara/graydon/graydon-moving/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (7,8,9,14,32,35,36,41,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
print('Creating SBI groups')
df = aggregate_transform.group_code_sbi(df)

Creating SBI groups


In [6]:
print('Creating code legal form groups')
df = aggregate_transform.group_code_legal_form(df)

Creating code legal form groups


In [7]:
print('Calculating sum of columns')
df = aggregate_transform.calculate_sum(df)

Calculating sum of columns


In [8]:
print('Calculating mean of columns')
df = aggregate_transform.calculate_mean(df)

Calculating mean of columns


In [9]:
print('Calculating ages')
df = aggregate_transform.calculate_age(df)

Calculating ages


In [61]:
aggregate_transform.key = ['id_branch', 'id_company', 'date_month']

In [34]:
list_column_pairs=[['qty_employees', 'delta_qty_employees'],
                   ['qty_issued_credit_reports', 'delta_qty_issued_credit_reports'],
                   ['score_payment_assessment', 'delta_score_payment_assessment'],
                   ['score_pd', 'delta_score_pd'],
                   ['code_legal_form', 'code_legal_form_has_changed'],
                   ['code_SBI_2_group', 'SBI_has_changed']]
subset_columns = aggregate_transform.key 
subset_columns.extend([item[0] for item in list_column_pairs])

In [35]:
subset_columns

['id_branch',
 'id_company',
 'date_month',
 'qty_employees',
 'qty_issued_credit_reports',
 'score_payment_assessment',
 'score_pd',
 'code_legal_form',
 'code_SBI_2_group']

In [39]:
temp_df = df.reset_index().loc[:, subset_columns].sort_values(aggregate_transform.key)

In [40]:
temp_df = temp_df.groupby(aggregate_transform.key_aggregation).agg(['first', 'last'])

In [53]:
# Calculating delta's
for column_pair in list_column_pairs:
    if temp_df[column_pair[0]]['first'].dtypes == 'object':
        temp_df[column_pair[1]] = temp_df[column_pair[0]]['last'] != temp_df[column_pair[0]]['first']
    else: 
        temp_df[column_pair[1]] = temp_df[column_pair[0]]['last'] - temp_df[column_pair[0]]['first']

float64
float64
float64
float64
float64
object


In [59]:
HTML(temp_df.head(900).to_html())

In [62]:
# Ungroup and deduplicate columns
#temp_df.columns = temp_df.columns.droplevel(1)
#temp_df = temp_df.loc[:,~temp_df.columns.duplicated()]
temp_df = temp_df.drop(axis=1, columns=[item[0] for item in list_column_pairs])

# Add delta's to original data-frame
df = df.merge(temp_df, how='left', on=['date_month', 'id_company', 'id_branch'])

In [63]:
df.head()

,Unnamed: 0,date_month,id_company,id_branch,date_established,is_discontinued,code_discontinuation,code_financial_calamity,date_financial_calamity_started,date_financial_calamity_stopped,...,years_since_last_amt_consolidated_revenue,years_since_last_amt_operating_result,years_since_last_qty_employees,years_since_last_amt_revenue,delta_qty_employees,delta_qty_issued_credit_reports,delta_score_payment_assessment,delta_score_pd,code_legal_form_has_changed,SBI_has_changed
0,0,2017-01-01,3,10079408,1921-03-17,False,NaN,NaN,NaN,NaN,...,2.0,8.0,0.0,8.0,0.0,-1.0,0.0,-0.035,NaN,False
1,1869701,2017-02-01,3,10079408,1921-03-17,False,NaN,NaN,NaN,NaN,...,2.0,8.0,0.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3745436,2017-03-01,3,10079408,1921-03-17,False,NaN,NaN,NaN,NaN,...,2.0,8.0,0.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
3,5625527,2017-04-01,3,10079408,1921-03-17,False,NaN,NaN,NaN,NaN,...,2.0,8.0,0.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7512392,2017-05-01,3,10079408,1921-03-17,False,NaN,NaN,NaN,NaN,...,2.0,8.0,0.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print('Calculating deltas')
df = aggregate_transform.calculate_deltas(df)

Calculating deltas


/home/mrodriguezlara/graydon/graydon-moving/lib/python3.5/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
print('Calculating variance of columns')
df = aggregate_transform.calculate_variance(df) 

In [ ]:
print('Calculating if any true')
df = aggregate_transform.set_true_if_any_true(df)

In [ ]:
print('Calculating ratios')
df = aggregate_transform.calculate_ratios(df)

In [ ]:
print('Creating counts from dummies')
df = aggregate_transform.count_dummies(df)

In [ ]:
print('Dropping old columns')
df = aggregate_transform.drop_superfluous_columns(df)

In [ ]:
print('Deduplicating rows of original dataframe')
df = aggregate_transform.deduplicate_rows(df)

In [ ]:
print('Setting NaN and Inf values')
df = aggregate_transform.impute_na_inf(df)